# Practico Integrador Modulo 4

Se pesentara un entorno Docker con Hadoop (HDFS) y la implementación de:  

Lo que haremos sera levantar un entorno de Docker con Hadoop usando diferentes tecnologias que aprendimos en el modulo


## Parte 1

Lo primero que debemos hacer es copiar el repositorio que contiene las herramientas tecnologicas para trabajar, el repositorio que utilizaremos es: https://github.com/lopezdar222/herramientas_big_data

Se nos otorga un archivo del tipo docker-compose-V1.yml con todas las configuraciones y conexiones entre si que son necesarias

Ejecutamos ese archivo con el comando | sudo docker-compose -f docker-compose-v1.yml up -d |:
* -f es para indicar que no usaremos el archivo por defecto si no el que nosotros poseemos con la configuracion especifica
* up es para levantar ese archivo
* -d para que se siga ejecutando en segundo plano y no nos tome la terminal es decir que la deje bloqueada.

![Composeup1](Imagenes/Ls.png)


En el github que hemos clonado tenemos una carpeta dataset cargado con archivos CSV, debemos copiarla dentro del contenedor namenode pero para copiarla primero vamos a crear una carpeta dentro del contenedor, para ello vamos a ejecutar el comando: |sudo docker cp Datasets namenode:/home/Datasets | donde: 
* cp significa copy (copiar) y le indicamos el nombre del archivo (Datasets)
* Luego indicamos el nombre del contenedor (namenode) y luego de los dos puntos le indicamos el directorio a donde tiene que copiarse

![cp_datasets](Imagenes/cp_datasets.png)

Luego crearemos un directorio en la raiz de hdfs (sistema de alamcenamiento de archivos de forma distribuida en hadoop), y copiamos los archivos CSV en esa carpeta

![hdfs](Imagenes/hdfs.png)

## Parte 2

En esta segunda parte trabajaremos con Hive, una plataforma de almacenamiento y procesamiento, la cual es parte del ecosistema Hadoop, para ello le daremos de alta al 2do archivo docker-compose para trabajar en ese entorno

Para crear las tablas dentro de hive debemos posicionarnos en el contenedor hive-server, pero primero debemos copiar de nuestro directorio en el servidor, el script con la creacion de todas las tablas, a continuacion esta la secuencia

![hive](Imagenes/hive.png)

Ejecutamos el script Paso02.hql con la creacion de todas las tablas, el comando es |hive -f Paso02.hql| hive iniciara el programa, -f indica que se va a proporcionar un archivo con comandos

![tablas](Imagenes/tablas_hive.png)

## Parte 3

Esta parte consiste en que las tablas que creamos anteriormente las podremos almacenar en formato Parquet(binario) ya que es una oportunidad de ocupar menos y tener mayor velocidad de lectura, luego se usara Snappy un programa que comprime y descomprime archivos mas rapido que sus programas alternativos

Se nos entrego un archivo con un script para que todo se ejecute de manera automatica, la modadalidad es la misma que en el paso anterior, copiaremos el archivo dentro del contenedor hive-server y lo ejecutaremos, luego mostraremos como es el script Paso03.hql
para ver que se estaria ejecutando

![paso03](Imagenes/paso03.png)

Se observa la creacion de una tabla tipo external, con sus columnas y tipos de dato, luego indica:  
* "STORED AS PARQUET" significa que los datos en la tabla se almacenaran en formato Parquet
* "LOCATION" nos da la ubicacion donde seran guardados los datos en HDFS, cabe recordar que las tablas external al ser eliminadas se elimina la tabla con su metadata pero no los datos
* "TBLPROPERTIES" es para asignar propiedades especificas a la tabla en este caso el tipo de compresion que se le aplicara a los datos para guardarlos y es Snappy

![asd](Imagenes/asd.png)

## Parte 4


En esta parte crearemos un indice para la tabla venta especificamente en la columna IdSucursal, el objetivos de los indices es de hacer un acceso mas veloz de la informacion y que el motor de base de datos no tenga que hacer escaneos completos de las tablas

Podemos observa que se selecciona la base de datos a utilizar, luego da un comando: 
* "CREATE INDEX" es la sentencia para crear un indice, luego indica el nombre del indice (index_venta_sucursal)
* "ON TABLE" se utiliza para señalar que tabla vamos a usar y entre parentesis se indica la columna
* "AS" lo que hace es especificar el manejador de indices
* "WITH DEFERRED REBUILD" lo que hace esta sentencia es decirle que no construya en ese momento el indice si no cuando se le indique, esto se hace para cuando se manejan grandes cantidades de datos y se debe de actualizar el indice para que haga las consultas con los nuevos datos ingresados

![indice](Imagenes/indice.png)

## Parte 5

Levantaremos un nuevo docker compose especifico para esta parte del practico, utilizaremos HBase, MongoDB, Neo4J y Zeppelin

Luego de dar de alta este nuevo compose, vamos ingresar al contenedor de Hbase y nos metemos en su interfaz de lineas de comando
* Crearemos una tabla (personal) y hbase nos pide que si o si al crear una tabla le demos una columna familiar (personal_Data)
* Le insertamos informacion de la siguiente forma:
  * put <nombre_de_tabla>
  * Indicamos "id"
  * Indicamos a personal_data sus componentes y los datos a insertar
* get lo usamos para traer la informacion, le indicamos la tabla y el "id"

### HBASE

![hbase1](Imagenes/hbase1.png)

Luego colocamos el archivo Personal.csv en hbase/data/personal.csv:
* -put es para indicar que el archivo (personal.csv) que esta en nuestro HDFS local se ubique en el HDFS de HBase

![copy_hbase](Imagenes/copy_hbase.png)

Con esto importamos el archivo csv y especificamos columnas

![importamos](Imagenes/importamos.png)

Aqui hacemos un escaneo de la tabla personal para corroborar que realizamos bien la importacion y creamos una nueva tabla con sus columnas familiares, le insertamos datos y realizamos de nuevo un get para ver los datos

![scaneo](Imagenes/scaneo.png)

![insertaalbumn](Imagenes/insertalbum.png)

### Mongo

Comenzaremos por copiar 2 archivos al contenedor de MongoDb

![copy_mongo](Imagenes/copy_mongo.png)

Nos ubicamos en el contenedor de Mongo e importamos los datos desde un archivo CSV y un archivo JSON en las colecciones "iris_csv" e "iris_json" en la base de datos "dataprueba" de MongoDB

![mongo2](Imagenes/mongo2.png)

Ingresamos al shell de Mongo, cambiamos de base de datos (dataprueba), verificamos que los archivos si se copiaron (Show collections) y luego los accedemos para ver la informacion dentro de ellos

![mongosh](Imagenes/mongosh.png)

Copiamos los archivos de la base de datos Mongo que nos dieron

![bdmon](Imagenes/bdmon.png)

Hago una observacion, falta crear un directorio en el contenedor namenode/tmp/udfs, ya que al ejectuar el hive -f iris.hql al ser ejecutado nos dira que no existe tal archivo y deberemos crear la carpeta udfs, ademas de copiar los archivos de mongo a ese directorio tambien

![udfs](Imagenes/udfs.png)

El ultimo punto de Mongo (8) inica el servidor de hive (hiveserver2) y le da permisos totales al archivo iris.hql, pero luego al momento de ejecutar da error, por ende hasta aqui fue realizado

## Parte 6

Trabajaremos con el Spark y Kafka

Iniciamos una sesion interactiva de PySpark

![spark1](Imagenes/spark1.png)

No me anduvo mas nada que eso🤯, lamento muchisimo no poder hacerlo, creo que con un poco de guia lo podria realizar completo, pero hay errores que nos los entiendo, falta de archivos, los coloco e igual tiran errores, hasta aqui he podido realizar, muchisimas gracias por la oportunidad, prometo dar mas esfuerzo para el proximo